## Delivery Truck Packing Problem 
In North America there are over 100 standard box sizes that consumer goods are shipped in.

Optimally loading delivery trucks with packages, expecially when additional considerations such as priority shipping status, order date and weight/size limits of the trucks are considered is a difficult combinatorial optimization problem.

### Objectives:
1. Maximize the number of packages selected with priority shipping
2. Minimize the number of days the packages are in transit

### Constraints:
1. Do not exceed the maximum number of packages that can fit on the truck (100)
2. Do not exceed the maximmum weight capacity of the truck (3000 lbs)

In [240]:
from dwave.system import LeapHybridCQMSampler
from dimod import ConstrainedQuadraticModel, Binary, quicksum
import random
import numpy as np

In [241]:
random.seed(6969)
num_items=300
priority=[random.choice([1, 2, 3]) for i in range(num_items)]
days_since_order=[random.choice([0, 1, 2, 3]) for i in range(num_items)]
cost=[random.randint(1, 100) for i in range(num_items)]  # weight
print(cost)

[73, 13, 91, 28, 56, 50, 50, 24, 58, 38, 75, 87, 80, 38, 4, 75, 29, 49, 100, 89, 12, 82, 45, 52, 19, 19, 9, 37, 52, 33, 31, 83, 39, 11, 69, 10, 56, 30, 98, 7, 24, 30, 34, 83, 87, 30, 27, 25, 91, 24, 21, 33, 44, 18, 10, 42, 21, 53, 82, 69, 2, 14, 83, 11, 87, 21, 90, 10, 50, 89, 23, 43, 33, 72, 42, 49, 90, 61, 57, 85, 1, 78, 88, 30, 6, 93, 94, 82, 66, 18, 65, 55, 49, 33, 23, 74, 96, 73, 21, 22, 92, 51, 3, 67, 97, 93, 71, 26, 1, 88, 63, 36, 18, 99, 25, 46, 95, 49, 55, 78, 97, 62, 70, 4, 61, 74, 19, 3, 11, 74, 46, 79, 39, 46, 29, 60, 21, 68, 75, 96, 9, 22, 46, 88, 33, 10, 83, 61, 71, 48, 80, 43, 79, 34, 42, 91, 54, 9, 56, 47, 8, 24, 79, 98, 96, 4, 40, 67, 81, 33, 98, 30, 57, 9, 100, 1, 35, 11, 30, 23, 33, 52, 48, 21, 87, 75, 19, 1, 69, 42, 82, 72, 81, 78, 12, 76, 86, 8, 50, 29, 59, 54, 64, 62, 95, 68, 25, 90, 94, 66, 40, 19, 21, 63, 27, 18, 19, 38, 89, 10, 96, 38, 43, 80, 29, 75, 23, 90, 10, 85, 93, 96, 37, 28, 70, 44, 38, 36, 18, 65, 9, 71, 20, 65, 21, 13, 88, 30, 4, 49, 95, 69, 37, 72, 1

In [242]:
max_weight=3000
max_parcels=100
obj_weight_priority=1.0
obj_weight_days=1

In [243]:
cqm=ConstrainedQuadraticModel()

In [244]:
x=[Binary(i) for i in range(num_items)]

#### Objective 1: Maximize priority shipping $ min(-\sum_{i=0}^{N}p_i\cdot x_i) $

In [245]:
obj1=-obj_weight_priority*quicksum(priority[i]*x[i] for i in range(num_items))

#### Objective 2: Minimize wait time $ min(-\sum_{i=0}^{N}d_i\cdot xi) $

In [246]:
obj2=-obj_weight_days*quicksum(days_since_order[i]*x[i] for i in range(num_items))

In [247]:
cqm.set_objective(obj1+obj2)

#### Constraint 1: Maximum parcels $ \sum_{i=0}^{N}x_i=P $

In [248]:
cqm.add_constraint(quicksum(x[i] for i in range(num_items)) <= max_parcels, label='max_parcels')

'max_parcels'

#### Constraint 2: Maximum capacity $ \sum_{i=0}^{N}w_i\cdot x_i\le W $

In [249]:
cqm.add_constraint(quicksum(cost[i]*x[i] for i in range(num_items)) <= max_weight, label='max_capacity')

'max_capacity'

In [250]:
cqm_sampler=LeapHybridCQMSampler()
samplest=cqm_sampler.sample_cqm(cqm, label='Truck Packing Demo', time_limit=10)

In [251]:
print(samplest.info)

{'constraint_labels': ['max_capacity', 'max_parcels'], 'qpu_access_time': 31844, 'charge_time': 10000000, 'run_time': 10082913, 'problem_id': 'fe140811-b1fd-4fbe-a8e2-a5392e0e025a', 'problem_label': 'Truck Packing Demo'}


In [252]:
feasible_sols=np.where(samplest.record.is_feasible==True)

In [253]:
feasible_sols[0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98])

In [254]:
import itertools

if len(feasible_sols[0]):
    first_feasible_sol=np.where(samplest.record[feasible_sols[0][0]][0]==1)
    #print(first_feasible_sol)
    
    problem_array=np.zeros((3, 4)).astype(int)
    for i in range(num_items):
        #print(-1*(priority[i]-3), -1*(days_since_order[i]-3))
        problem_array[-1*(priority[i]-3)][-1*(days_since_order[i]-3)]+=1
    #print(problem_array)
    print("\n****************** PROBLEM ******************")
    print("\tDays since order was placed")
    print("{:>5s}{:>5s}{:^10s}{:^4s}{:^5s}".format("Priority |", "3", "2", "1", "0"))
    print("-" * 40)
    for i in range(3):
        print("{:>5s}{:>10s}{:^10s}{:^5s}{:^5s}".format(str(-1*(i-3)), str(problem_array[i][0]), str(problem_array[i][1]), str(problem_array[i][2]), str(problem_array[i][3])))
        
    solution_array=np.zeros((3, 4)).astype(int)
    total_weight=0
    for i in range(len(first_feasible_sol[0])):
        solution_array[-1*(priority[first_feasible_sol[0][i]]-3)][-1*(days_since_order[first_feasible_sol[0][i]]-3)]+=1
        total_weight+=cost[first_feasible_sol[0][i]]
    print("\n****************** SOLUTION ******************")
    print("\tDays since order was placed")
    print("{:>5s}{:>5s}{:^10s}{:^4s}{:^5s}".format("Priority |", "3", "2", "1", "0"))
    print("-" * 40)
    for i in range(3):
        print("{:>5s}{:>10s}{:^10s}{:^5s}{:^5s}".format(str(-1*(i-3)), str(solution_array[i][0]), str(solution_array[i][1]), str(solution_array[i][2]), str(solution_array[i][3])))
        
    #first=next(itertools.filterfalse(lambda d: not getattr(d, 'is_feasible'), list(samplest.data())))
    #print(first)
    infeasible_sample=next(itertools.filterfalse(lambda d: not getattr(d, 'is_feasible'), list(samplest.data())))
    print(infeasible_sample.is_satisfied)
        
    print("\nTotal number of selected items:", len(first_feasible_sol[0]))
    print("Total weight of selected items:", total_weight)


****************** PROBLEM ******************
	Days since order was placed
Priority |    3    2      1    0  
----------------------------------------
    3        30    20     23   16  
    2        33    29     28   23  
    1        21    31     25   21  

****************** SOLUTION ******************
	Days since order was placed
Priority |    3    2      1    0  
----------------------------------------
    3        25    14     11    3  
    2        22    13      1    0  
    1         9    2       0    0  
[ True  True]

Total number of selected items: 100
Total weight of selected items: 2997


#### just playground

In [255]:
feasible_samplest=samplest.filter(lambda row: row.is_feasible)

if not len(feasible_samplest):
    raise ValueError("No feasible solution found")
    
best=feasible_samplest.first

selected_item_indices=[key for key, val in best.sample.items() if val==1.0]

print("\nFound best solution at energy {}".format(best.energy))
print("\nSelected item numbers (0-indexed):", selected_item_indices)


Found best solution at energy -481.0

Selected item numbers (0-indexed): [1, 5, 6, 7, 11, 14, 15, 16, 20, 24, 26, 27, 30, 35, 36, 37, 43, 45, 46, 47, 49, 50, 52, 56, 58, 60, 63, 65, 70, 71, 80, 84, 89, 98, 99, 102, 107, 108, 109, 111, 112, 114, 115, 123, 127, 128, 132, 134, 136, 140, 141, 144, 147, 149, 153, 154, 157, 158, 171, 172, 175, 177, 178, 179, 180, 183, 186, 187, 188, 189, 191, 194, 197, 199, 200, 201, 205, 206, 212, 215, 226, 236, 238, 242, 245, 248, 254, 259, 261, 262, 263, 274, 276, 279, 280, 281, 288, 293, 294, 297]


In [256]:
feasible_sampleset=samplest.filter(lambda d: d.is_feasible)
num_feasible=len(feasible_sampleset)
if num_feasible>0:
    best_samples=feasible_sampleset.truncate(min(10, num_feasible))
else:
    warnings.warn("Warning: Did not find feasible solution")
    best_samples=samplest.truncate(10)
    
print(" \n"+"="*30+"BEST SAMPLE SET"+"="*30)
print(best_samples)

best_sample=best_samples.first.sample
#print(best_sample)

 
==============================BEST SAMPLE SET==============================
    0   1   2   3   4   5   6   7   8   9  10  11 ... 299 energy num_oc. ...
0 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 ... 0.0 -481.0       1 ...
1 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 ... 0.0 -481.0       1 ...
2 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 ... 0.0 -481.0       1 ...
3 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 ... 0.0 -481.0       1 ...
4 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 ... 0.0 -481.0       1 ...
5 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 ... 0.0 -481.0       1 ...
6 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 0.0 ... 0.0 -481.0       1 ...
7 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 ... 0.0 -481.0       1 ...
8 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 ... 0.0 -481.0       1 ...
9 0.0 1.0 0.0 0.0 0.0 1.0 1.0 1.0 0.0 0.0 0.0 1.0 ... 0.0 -481.0       1 ...
['INTEGER', 10 rows, 10 samples, 300 variables]
